In [1]:
import sagemaker

# Get a SageMaker-compatible role used by this Notebook Instance.
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

# S3 prefix
prefix = 'scikit-iris'

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn import datasets

# Load Iris dataset, then join labels and features together
iris = datasets.load_iris()
joined_iris = np.insert(iris.data, 0, iris.target, axis=1)

df = pd.DataFrame(joined_iris, columns = ['label','sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)'])
train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))])

# Split testing data and label for later model deploy test
test_y = test_data.iloc[:,0]
test_X = test_data.iloc[:,1:]

# Create a temporary directory and write the dataset as CSV
os.makedirs('./data', exist_ok=True)
#np.savetxt('./data/iris.csv', joined_iris, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')
train_data.to_csv('data/train.csv', index=False, header=False)
validation_data.to_csv('data/validation.csv', index=False, header=False)
test_X.to_csv('data/test.csv', index=False, header=False)

In [4]:
# Upload the dataset to S3
s3_input_train = sess.upload_data('data/train.csv', bucket=bucket, key_prefix=f'{prefix}/data')
s3_input_validation = sess.upload_data('data/validation.csv', bucket=bucket, key_prefix=f'{prefix}/data')
s3_input_test = sess.upload_data('data/test.csv', bucket=bucket, key_prefix=f'{prefix}/test')

## Training

The below script contains both training and inference functionality and can run both in SageMaker Training hardware or locally (desktop, SageMaker notebook, on prem, etc). Detailed guidance here https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#train-a-model-with-scikit-learn

In [5]:
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn('train.py', #/opt/ml/code/<script file>
                  instance_type="ml.c5.xlarge",
                  instance_count=1,
                  framework_version='0.20.0',
                  py_version='py3',
                  role=role,
                  sagemaker_session=sess,
                  metric_definitions=[{"Name": "model_accuracy", 
                                       "Regex": "Model Accuracy: ([0-9.]+).*$"}
                                     ],
                  hyperparameters={'max_leaf_nodes': 30} 
                  #/opt/ml/input/config/hyperparameters.json
                )

sklearn_estimator.fit({"train": s3_input_train, "test": s3_input_validation})

2022-06-14 07:37:16 Starting - Starting the training job...
2022-06-14 07:37:40 Starting - Preparing the instances for trainingProfilerReport-1655192236: InProgress
......
2022-06-14 07:38:45 Downloading - Downloading input data
2022-06-14 07:38:45 Training - Downloading the training image...
2022-06-14 07:39:11 Training - Training image download completed. Training in progress..2022-06-14 07:39:13,067 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-06-14 07:39:13,070 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-14 07:39:13,077 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-06-14 07:39:13,400 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-14 07:39:13,413 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-14 07:39:13,422 sagemaker-training-toolkit INFO     No GPUs detected (normal i

## Deploy an Endpoint 

Now that we've trained the algorithm on our data, let's deploy a model that's hosted behind a real-time endpoint.

In [6]:
predictor = sklearn_estimator.deploy(initial_instance_count=1, instance_type="ml.m5.large")

-----!

In [7]:
print("Predictions: {}".format(predictor.predict(test_X.values)))
print("Actual: {}".format(test_y.values))

Predictions: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 1. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]
Actual: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 2. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]


In [8]:
predictor.delete_endpoint(delete_endpoint_config=True)

## Deploy an Endpoint from Model Data (Optional)

In [9]:
# Download the trained model data from S3
from sagemaker.s3 import S3Downloader

s3_model_path = sklearn_estimator.model_data

S3Downloader.download(s3_uri=s3_model_path,
                          local_path='./',
                          sagemaker_session=sess)

#or

#import boto3

#s3_client = boto3.client('s3')
#training_job_name = sklearn_estimator.latest_training_job.name

#with open('model.tar.gz', 'wb') as data:
#    s3_client.download_fileobj(Bucket=bucket, Key=f'{training_job_name}/output/model.tar.gz', Fileobj=data)

!tar -zxvf model.tar.gz

model.joblib


### Model Directory Structure

The contents of model.tar.gz should be organized as follows:

- Model files in the top-level directory
- Inference script (and any other source files) in a directory named code/ (for more about the inference script, see The SageMaker Scikit-learn Model Server)
- Optional requirements file located at code/requirements.txt (for more about requirements files, see Using third-party libraries)

<b>model.tar.gz</b>
- model.joblib
- code
     - inference.py
     - requirements.txt

### Inference Script function definition 
- <b>model_fn:</b> Load the model file in model directory (eg. /opt/ml/model/model.pth).
- <b>input_fn:</b> Deserialize the Invoke request body into an object we can perform prediction on.
- <b>predict_fn:</b> Perform prediction on the deserialized object, with the loaded model.
- <b>output_fn:</b> Serialize the prediction result into the desired response content type.

https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#load-a-model

In [10]:
# Package model data and inference script to model.tar.gz
!mkdir code
!cp inference.py code/
!tar -czvf sklearn-model.tar.gz model.joblib code
model_path = sess.upload_data(path='sklearn-model.tar.gz', key_prefix=f'{prefix}/models')

mkdir: cannot create directory ‘code’: File exists
model.joblib
code/
code/inference.py


In [11]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(model_data=model_path,
                     framework_version='0.20.0',
                     py_version='py3',
                     role=role,
                     sagemaker_session=sess,
                     entry_point='code/inference.py',
                    )
predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.large")

-----!

In [12]:
print("Predictions: {}".format(predictor.predict(test_X.values)))
print("Actual: {}".format(test_y.values))

Predictions: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 1. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]
Actual: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 2. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]


In [13]:
predictor.delete_endpoint(delete_endpoint_config=True)

## Batch Transform

In [14]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_estimator.transformer(instance_count=1, instance_type='ml.m5.large')

In [15]:
# Start a transform job and wait for it to finish
transformer.transform(s3_input_test, content_type='text/csv')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()

.........................Processing /opt/ml/code
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for sklearn-iris: filename=sklearn_iris-1.0.0-py2.py3-none-any.whl size=5407 sha256=dca24670833023c344aabe7927729b89f8aa8b5fb2844235c44e1361e3b45f37
  Stored in directory: /tmp/pip-ephem-wheel-cache-cg7910q4/wheels/3e/0f/51/2f1df833dd0412c1bc2f5ee56baac195b5be563353d111dca6
Successfully built sklearn-iris
/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
 

In [16]:
# Download the output data from S3 to local filesystem
batch_output = transformer.output_path
!aws s3 cp --recursive $batch_output/ batch_data/

download: s3://sagemaker-ap-east-1-468208999430/sagemaker-scikit-learn-2022-06-14-07-45-02-164/test.csv.out to batch_data/test.csv.out


## Automatic model Tuning (optional)

In [19]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {"max_leaf_nodes": IntegerParameter(5, 100)}
objective_metric_name = 'model_accuracy'

tuner = HyperparameterTuner(sklearn_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions=[{"Name": "model_accuracy", 
                                                 "Regex": "Model Accuracy: ([0-9.]+).*$"}],
                            objective_type='Maximize',
                            max_jobs=9,
                            max_parallel_jobs=3)

tuner.fit({'train': s3_input_train, 'test': s3_input_validation}, wait=False) #wait=False to set async training job

In [20]:
# Wait for the Hyperparameter Tuning Job to be completed

tuner.wait()

#or

#tuner.logs()

#or

#tuning_job_name = sklearn_estimator.latest_training_job.name
#sess.wait_for_tuning_job(tuner.latest_tuning_job.name)

............................................................................................!


In [21]:
# return the best training job name
tuner.best_training_job()

'sagemaker-scikit-lea-220614-1026-002-281e67ea'

In [22]:
# Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')


2022-06-14 10:29:00 Starting - Preparing the instances for training
2022-06-14 10:29:00 Downloading - Downloading input data
2022-06-14 10:29:00 Training - Training image download completed. Training in progress.
2022-06-14 10:29:00 Uploading - Uploading generated training model
2022-06-14 10:29:00 Completed - Training job completed
----!

In [23]:
# Deploy the best one and predict
print("Predictions: {}".format(tuner_predictor.predict(test_X.values)))
print("Actual: {}".format(test_y.values))

Predictions: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 1. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]
Actual: [1. 0. 1. 1. 0. 1. 2. 2. 0. 1. 2. 2. 0. 2. 0. 1. 2. 2. 1. 2. 1. 1. 2. 2.
 0. 1. 2. 0. 1. 2.]


In [24]:
tuner_predictor.delete_endpoint(delete_endpoint_config=True)